In [18]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer, QuantileTransformer, PowerTransformer, MaxAbsScaler, PolynomialFeatures

In [80]:
# specify the directory containing the data
train = pd.read_csv(r'C:\Users\55219\Desktop\Competicoes\kaggle competitions download -c playground-series-s3e5\train.csv')
test = pd.read_csv(r'C:\Users\55219\Desktop\Competicoes\kaggle competitions download -c playground-series-s3e5\test.csv')
wineqt = train = pd.read_csv(r'C:\Users\55219\Desktop\Competicoes\kaggle competitions download -c playground-series-s3e5\WineQT.csv')

# Retirando a coluna Id.
train.drop(columns=['Id'], inplace=True)
train = train.drop_duplicates()
 
#train_wineqt = pd.concat([train, wineqt], axis=0)
#train_wineqt = train_wineqt.drop_duplicates()

#Features that will be using to train the model.
features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide',
             'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

X = train[features].copy()
y = train['quality'].copy()
 
#Applying the logarithmic function in the column "free sulfur dioxide", improved my metric a little bit
X['free sulfur dioxide'] = np.sqrt(X['free sulfur dioxide'])


kf = KFold(n_splits=11, random_state=31, shuffle=True)

lista_ck = []
lista_acc = []

#Implementing Cross validation so I can have more freedom to do thing inside it
for tr, ts in kf.split(X, y):
    Xtr, Xval = X.iloc[tr], X.iloc[ts]
    ytr, yval = y.iloc[tr], y.iloc[ts]
    
    #Creating a Scaler
    scaler = StandardScaler()
    #Xtr = scaler.fit_transform(Xtr)
    #Xval = scaler.fit_transform(Xval)

    #Creating the model
    model = RandomForestClassifier(n_estimators=400, max_depth=30, random_state=31)
    #model = LGBMClassifier(n_estimators=200, learning_rate=0.322, max_depth=3 , random_state=31)

    #Training the model.
    model.fit(Xtr, ytr)

    #Making predictions
    pdt_ = model.predict(Xval)
    #The metric for this competition is "Cohen Kappa" 
    ck = cohen_kappa_score(yval, pdt_)
    acc = accuracy_score(yval, pdt_)
    #print(Xtr.shape)
    #print(Xval.shape)
    #print(ytr.shape)
    #print(yval.shape)
    print('Cohen Kappa: {:.3f} - Accuracy: {:.3f}'.format(ck, acc))
    lista_ck.append(ck)
    lista_acc.append(acc)
print('')
print('Cohan Kappa Médio: {:.3f}'.format(np.mean(lista_ck)))
print('Cohan Kappa Desvio Padrão: {:.3f}'.format(np.std(lista_ck)))

print('Accuracy Médio: {:.3f}'.format(np.mean(lista_acc)))
#primeiro cohen_kappa com dt e arvore de decisão - 0.19782921563924194
print('')


Cohen Kappa: 0.446 - Accuracy: 0.656
Cohen Kappa: 0.265 - Accuracy: 0.570
Cohen Kappa: 0.301 - Accuracy: 0.570
Cohen Kappa: 0.293 - Accuracy: 0.581
Cohen Kappa: 0.388 - Accuracy: 0.645
Cohen Kappa: 0.509 - Accuracy: 0.710
Cohen Kappa: 0.265 - Accuracy: 0.554
Cohen Kappa: 0.335 - Accuracy: 0.576
Cohen Kappa: 0.283 - Accuracy: 0.543
Cohen Kappa: 0.351 - Accuracy: 0.630
Cohen Kappa: 0.553 - Accuracy: 0.717

Cohan Kappa Médio: 0.363
Cohan Kappa Desvio Padrão: 0.095
Accuracy Médio: 0.614



# Observations:
- There are a few duplicated rows in our original dataset, that causes data leakage.